In [1]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from sklearn.linear_model import RidgeClassifier

In [2]:
emp_df=pd.read_csv('/content/processed_emp_data.csv')
emp_df.head()

,Age,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,WorkLifeBalance,YearsAtCompany,Attrition
0,41,446,1,2,606,2,4,470,5993,8,1,6,1
1,49,961,8,1,606,3,2,673,5130,1,3,10,0
2,37,961,2,2,82,4,3,470,2090,6,3,0,1
3,33,961,3,4,606,4,3,673,2909,1,3,8,0
4,27,961,2,1,464,1,2,673,3468,9,3,2,0


In [3]:
# identifying X and y
X=emp_df.drop(['Attrition'],axis=1)
y=emp_df['Attrition']

In [4]:
# define models and parameters
model = RidgeClassifier()
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# define grid search
grid = dict(alpha=alpha)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.839456 using {'alpha': 0.1}
0.839456 (0.003333) with: {'alpha': 0.1}
0.839456 (0.003333) with: {'alpha': 0.2}
0.839456 (0.003333) with: {'alpha': 0.3}
0.839456 (0.003333) with: {'alpha': 0.4}
0.839456 (0.003333) with: {'alpha': 0.5}
0.839456 (0.003333) with: {'alpha': 0.6}
0.839456 (0.003333) with: {'alpha': 0.7}
0.839456 (0.003333) with: {'alpha': 0.8}
0.839456 (0.003333) with: {'alpha': 0.9}
0.839456 (0.003333) with: {'alpha': 1.0}


In [5]:
# lets cross verfiy the result using train test split approach
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=21)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1176, 12)
(294, 12)
(1176,)
(294,)


In [6]:
# lets fit the model with the data
ridge_model=RidgeClassifier(alpha=0.1)
ridge_model.fit(X_train,y_train)

RidgeClassifier(alpha=0.1)

In [7]:
# prediction on y
y_train_pred=ridge_model.predict(X_train)
y_test_pred=ridge_model.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
#accuracy check on train and test data
print('accuracy score of training data for ridge classifier: ',accuracy_score(y_train,y_train_pred))
print('accuracy score of testing data for ridge classifier: ',accuracy_score(y_test,y_test_pred))

accuracy score of training data for ridge classifier:  0.8384353741496599
accuracy score of testing data for ridge classifier:  0.8401360544217688


In [9]:
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import roc_auc_score
clf = RidgeClassifierCV().fit(X, y)
roc_auc_score(y, clf.decision_function(X), average=None)

0.7265254721597695